In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import networkx as nx
from tqdm.notebook import tqdm
import gc
import pickle
import random

In [ ]:
def load_file(file):

    r = open(file,'rb')
    x = pickle.load(r)
    r.close()

    return x

In [ ]:
def save_file(x,file):

    r = open(file,'wb')
    pickle.dump(x,r)

In [ ]:
def degree_distribution(F):
    degree = pd.DataFrame(nx.degree(F))
    degree = degree.sort_values(by=[1], ascending=False)
    citypdf_sum2 = pd.DataFrame(degree[1].value_counts())
    citypdf_sum2['f'] = citypdf_sum2.index
    citypdf_sum2['频率'] = citypdf_sum2[1]/citypdf_sum2[1].sum()
    citypdf_sum2 = citypdf_sum2.sort_values(by=['f'], ascending=True)
    ccdf_sum2 = np.zeros(len(citypdf_sum2))
    for i in range(len(ccdf_sum2)):
        ccdf_sum2[i] = 1-np.sum(citypdf_sum2.values[0:i,2])
    citypdf_sum2['ccdf'] = ccdf_sum2
    gdo = pd.DataFrame(F.out_degree())
    gdo = gdo.sort_values(by=[1], ascending=False)
    citypdf_sum = pd.DataFrame(gdo[1].value_counts())
    citypdf_sum['f'] = citypdf_sum.index
    citypdf_sum['频率'] = citypdf_sum[1]/citypdf_sum[1].sum()
    citypdf_sum = citypdf_sum.sort_values(by=['f'], ascending=True)
    ccdf_sum = np.zeros(len(citypdf_sum))
    for i in range(len(ccdf_sum)):
        ccdf_sum[i] = 1-np.sum(citypdf_sum.values[0:i,2])
    citypdf_sum['ccdf'] = ccdf_sum
    gdi = pd.DataFrame(F.in_degree())
    gdi = gdi.sort_values(by=[1], ascending=False)
    citypdf_sum1 = pd.DataFrame(gdi[1].value_counts())
    citypdf_sum1['f'] = citypdf_sum1.index
    citypdf_sum1['频率'] = citypdf_sum1[1]/citypdf_sum1[1].sum()
    citypdf_sum1 = citypdf_sum1.sort_values(by=['f'], ascending=True)
    ccdf_sum1 = np.zeros(len(citypdf_sum1))
    for i in range(len(ccdf_sum1)):
        ccdf_sum1[i] = 1-np.sum(citypdf_sum1.values[0:i,2])
    citypdf_sum1['ccdf'] = ccdf_sum1
    degree.set_index([0], inplace=True)
    gdo.set_index([0], inplace=True)
    gdi.set_index([0], inplace=True)
    degree_all = pd.concat([degree, gdo, gdi], axis=1)
    degree_all.columns = ['Degree','Out-degree','In-degree']
    return degree_all, citypdf_sum2, citypdf_sum, citypdf_sum1

In [ ]:
list_all = pd.read_excel('F:\\dataset\\mpeuni\\Theorems\\new\\list_all.xlsx', header = 0)
adjacency_listT = pd.read_excel('F:\\dataset\\mpeuni\\Theorems\\new\\adjacency_list_Theorems_and_Axioms_only.xlsx', header = 0)
FTheorem = nx.from_pandas_edgelist(adjacency_listT, 'Theorem_id', 'Ref_id', create_using=nx.DiGraph())

In [ ]:
nodelistor = list(FTheorem)[:100]

In [ ]:
F = FTheorem.subgraph(nodelistor)

In [ ]:
F = nx.convert_node_labels_to_integers(F, first_label=0, ordering='default', label_attribute=None)

In [ ]:
vectoror = np.random.randint(0,2,(100,10))

In [ ]:
#theorem

In [ ]:
p = 0.05
w = 0.5
a = 2

In [ ]:
for i in tqdm(range(26326)):
    possible_neighbors = list(F.nodes)
    F.add_node(i + 100)
    vectori = np.random.randint(0, 2, (1, 10))[0]
    norm_vectori = np.linalg.norm(vectori)
    cos_sim = np.dot(vectoror, vectori) / (np.linalg.norm(vectoror, axis=1) * norm_vectori)
    cos_simcount1 = cos_sim > w
    possible_neighbors1 = [possible_neighbors[n] for n in np.where(cos_simcount1)[0]]
    degrees1 = [F.in_degree(n) for n in possible_neighbors1]
    new_degrees1 = np.array(degrees1) + a
    vectoror = np.vstack((vectoror, vectori))
    
    if len(possible_neighbors1) == 0:
        continue
    
    k = 0
    while k < len(possible_neighbors):
        j1 = random.choices(possible_neighbors1, weights=new_degrees1, k=1)[0]
        F.add_edge(i + 100, j1)
        P = np.random.binomial(1, p)
        if P == 1:
            k += 1
            break
        else:
            k += 1

In [ ]:
#mathematical paper

In [ ]:
p = 0.37
a = 2
w = 0.85
z = 0.028

In [ ]:
lc_count = []
sc_count = []

In [ ]:
for i in tqdm(range(38928)):
    possible_neighbors = list(F.nodes)
    F.add_node(i + 100)
    vectori = np.random.randint(0, 2, (1, 10))[0]
    norm_vectori = np.linalg.norm(vectori)
    cos_sim = np.dot(vectoror, vectori) / (np.linalg.norm(vectoror, axis=1) * norm_vectori)
    cos_simcount1 = cos_sim > w
    possible_neighbors1 = [possible_neighbors[n] for n in np.where(cos_simcount1)[0]]
    degrees1 = [F.in_degree(n) for n in possible_neighbors1]
    new_degrees1 = np.array(degrees1) + a
    vectoror = np.vstack((vectoror, vectori))
    
    if len(possible_neighbors1) == 0:
        continue
    
    k = 0
    number = 0
    number1 = 0
    while k < len(possible_neighbors):
        j1 = random.choices(possible_neighbors1, weights=new_degrees1, k=1)[0]
        F.add_edge(i + 100, j1)
        number = number + 1
        choices_neighbors1 = list(F.successors(j1))
        choices_neighbors2 = list(F.predecessors(j1))
        choices_neighbors2.remove(i + 100)
        choices_neighbors = choices_neighbors1 + choices_neighbors2
        X = np.random.binomial(1, z, len(choices_neighbors))
        linklist1 = np.where(X == 1)[0]
        number1 = number1 + len(linklist1)
        for n in linklist1:
            F.add_edge(i + 100, choices_neighbors[n])
        P = np.random.binomial(1, p)
        if P == 1:
            k += 1
            break
        else:
            k += 1
    lc_count.append(number)
    sc_count.append(number1)

In [ ]:
#paper

In [ ]:
p = 0.65
a = 1
w = 0.8
z = 0.125

In [ ]:
lc_count = []
sc_count = []

In [ ]:
for i in tqdm(range(27300)):
    possible_neighbors = list(F.nodes)
    F.add_node(i + 100)
    vectori = np.random.randint(0, 2, (1, 10))[0]
    norm_vectori = np.linalg.norm(vectori)
    cos_sim = np.dot(vectoror, vectori) / (np.linalg.norm(vectoror, axis=1) * norm_vectori)
    cos_simcount1 = cos_sim > w
    possible_neighbors1 = [possible_neighbors[n] for n in np.where(cos_simcount1)[0]]
    degrees1 = [F.in_degree(n) for n in possible_neighbors1]
    new_degrees1 = np.array(degrees1) + a
    vectoror = np.vstack((vectoror, vectori))
    
    if len(possible_neighbors1) == 0:
        continue
    
    k = 0
    number = 0
    number1 = 0
    while k < len(possible_neighbors):
        j1 = random.choices(possible_neighbors1, weights=new_degrees1, k=1)[0]
        F.add_edge(i + 100, j1)
        number = number + 1
        choices_neighbors1 = list(F.successors(j1))
        choices_neighbors2 = list(F.predecessors(j1))
        choices_neighbors2.remove(i + 100)
        choices_neighbors = choices_neighbors1 + choices_neighbors2
        X = np.random.binomial(1, z, len(choices_neighbors))
        linklist1 = np.where(X == 1)[0]
        number1 = number1 + len(linklist1)
        for n in linklist1:
            F.add_edge(i + 100, choices_neighbors[n])
        P = np.random.binomial(1, p)
        if P == 1:
            k += 1
            break
        else:
            k += 1
    lc_count.append(number)
    sc_count.append(number1)

In [ ]:
F.number_of_nodes()

In [ ]:
F.number_of_edges()

In [ ]:
gdo = pd.DataFrame(F.out_degree())
gdo = gdo.sort_values(by=[1], ascending=False)

In [ ]:
gdo

In [ ]:
gdi = pd.DataFrame(F.in_degree())
gdi = gdi.sort_values(by=[1], ascending=False)

In [ ]:
gdi

In [ ]:
degree_all, degreepdf, outdegreepdf, indegreepdf = degree_distribution(F)

In [ ]:
fig = plt.figure(figsize=(5, 5))
grid = plt.GridSpec(1, 1)
plt.rcParams['xtick.direction'] = 'in'
plt.rcParams['ytick.direction'] = 'in'

ax2 = fig.add_subplot(grid[0:1, 0:1])
ax2.set_xscale('log')
ax2.set_yscale('log')
ax2.plot(degreepdf['f'], degreepdf['ccdf'], '-o', color='#1F77B4', markersize=3, label='$k$')
ax2.plot(outdegreepdf['f'], outdegreepdf['ccdf'], '-o', color='#175987', markersize=3,label='$k^{out}$')
ax2.plot(indegreepdf['f'], indegreepdf['ccdf'], '-o', color='#6092B4', markersize=3,label='$k^{in}$')
ax2.set_ylabel('Cumulative probability', fontsize=14)
ax2.set_xlabel('The degree of citation network', fontsize=14)
ax2.legend(frameon=False)

In [ ]:
GMresults = [p,a,w,z,F,lc_count,sc_count]

In [ ]:
save_file(GMresults,'生成模型FcitGM-01.pkl')